In [1]:
# import libraries
#import boto3, re, sys, math, json, os, sagemaker, 
import urllib.request
import numpy as np                                
import pandas as pd  
import matplotlib
import matplotlib.pyplot as plt  
from IPython.display import Image                 
from IPython.display import display               
from time import gmtime, strftime       
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import neighbors
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.mixture import GaussianMixture
#from sagemaker.predictor import csv_serializer  
#from sagemaker import get_execution_role

# Global variables
Data_Download_Completed = True
Use_Small_DataSet = False
Small_Data_Already_Split = True
Dominate_Component_Number = 10
Feature_Weight_Step_Threshold = 0.01

/home/will/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Grab the necessary dataset files from the origin sources
data_dir="/home/will/Desktop/CSC8515/"
unsw_nb15_mainurl = 'https://www.unsw.adfa.edu.au/unsw-canberra-cyber/cybersecurity/ADFA-NB15-Datasets/'
unsw_nb15_training_file = 'UNSW_NB15_training-set.csv'
unsw_nb15_feature_file = 'NUSW-NB15_features.csv' # misspelling in the origin data file name
unsw_nb15_test_file = 'UNSW_NB15_testing-set.csv'
download_training_url = unsw_nb15_mainurl + unsw_nb15_training_file
download_feature_url = unsw_nb15_mainurl + unsw_nb15_feature_file
download_test_url = unsw_nb15_mainurl + unsw_nb15_test_file

if(Data_Download_Completed !=True):
    try:
      urllib.request.urlretrieve (download_feature_url, unsw_nb15_feature_file)
      print('Success: ' + unsw_nb15_feature_file)
    except Exception as e:
      print('Data load error: ',e)

    try:
      urllib.request.urlretrieve (download_training_url, unsw_nb15_training_file)
      print('Success: ' + unsw_nb15_training_file)
    except Exception as e:
      print('Data load error: ',e)


In [3]:
# Load data
# Must declare data_dir as the directory of training and test files
raw_data_filename = data_dir + unsw_nb15_training_file
if(Use_Small_DataSet !=True):
    print('Loading raw data')
    try:
        raw_data = pd.read_csv(raw_data_filename, index_col=0, header=0)
        print('Success: Data loaded into dataframe.')
    except Exception as e:
        print('Data load error: ',e)
    selected_data = raw_data
    print(selected_data.shape)


Loading raw data
Success: Data loaded into dataframe.
(82332, 44)


In [4]:

# Split smaller dataset for experimentation
samll_filename = data_dir + 'UNSW_NB15_training-small.csv'
if((Use_Small_DataSet ==True) and (Small_Data_Already_Split !=True)):
    remaining_data, selected_data = np.split(raw_data.sample(frac=1, random_state=1729), [int(0.80 * len(raw_data))])
    print(remaining_data.shape, selected_data.shape)
    selected_data.head(10)
    # Write the samll data set to disk
    selected_data.to_csv(samll_filename)

In [5]:
# Read back the small data
if(Use_Small_DataSet ==True):
    try:
        small_data = pd.read_csv(samll_filename, index_col=0, header=0)
        print('Success: Data loaded into dataframe.')
    except Exception as e:
        print('Data load error: ',e)
    selected_data = small_data
    print(selected_data.shape)


In [6]:
    selected_data.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
1,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,...,1,2,0,0,0,1,2,0,Normal,0
2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,...,1,2,0,0,0,1,2,0,Normal,0
3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,...,1,3,0,0,0,1,3,0,Normal,0
4,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,...,1,3,0,0,0,2,3,0,Normal,0
5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,...,1,3,0,0,0,2,3,0,Normal,0


In [7]:
# Now gather the list of feature names
print('Loading feature list')
try:
    feature_data = pd.read_csv(raw_data_filename, index_col=0, header=None, nrows=1)
    print('Success: Feature list loaded into dataframe.')
except Exception as e:
    print('Data load error: ',e)
#print(feature_data)
feature_list = feature_data.values[0]
feature_only_list = np.delete(feature_list, [(feature_list.shape[0]-2), (feature_list.shape[0]-1)])
print(feature_only_list.shape)
feature_only_list

Loading feature list
Success: Feature list loaded into dataframe.
(42,)


array(['dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss',
       'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb',
       'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean',
       'trans_depth', 'response_body_len', 'ct_srv_src', 'ct_state_ttl',
       'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd',
       'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports'], dtype=object)

In [8]:
RFE_feature_list = ['state', 'dttl', 'dload', 'swin', 'dwin', 'synack', 'ct_state_ttl', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'ct_srv_dst', 'proto', 'dbytes', 'dloss', 'ct_src_dport_ltm', 'spkts', 'sbytes', 'sttl', 'ct_srv_src', 'sjit', 'dmean', 'sloss', 'smean', 'trans_depth', 'dur']
Ultra_short_feature_list = ['ct_state_ttl', 'dbytes', 'dmean', 'dttl', 'dur', 'sbytes', 'state', 'sttl', 'synack']

In [9]:
print('Transforming data')
# Factorize columns: "proto", "service", "state", "attack_cat"
selected_data['proto'], protocols = pd.factorize(selected_data['proto'])
selected_data['service'], services = pd.factorize(selected_data['service'])
selected_data['state'], states    = pd.factorize(selected_data['state'])
#selected_data['attack_cat'], attacks = pd.factorize(selected_data['attack_cat'])
selected_data.head()

Transforming data


,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
id,,,,,,,,,,,,,,,,,,,,,
1,0.000011,0,0,0,2,0,496,0,90909.0902,254,...,1,2,0,0,0,1,2,0,Normal,0
2,0.000008,0,0,0,2,0,1762,0,125000.0003,254,...,1,2,0,0,0,1,2,0,Normal,0
3,0.000005,0,0,0,2,0,1068,0,200000.0051,254,...,1,3,0,0,0,1,3,0,Normal,0
4,0.000006,0,0,0,2,0,900,0,166666.6608,254,...,1,3,0,0,0,2,3,0,Normal,0
5,0.000010,0,0,0,2,0,2126,0,100000.0025,254,...,1,3,0,0,0,2,3,0,Normal,0


In [10]:
selected_data.describe()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,label
count,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,8.233200e+04,8.233200e+04,8.233200e+04,82332.000000,...,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000,82332.000000
mean,1.006756,8.811216,1.901739,0.862046,18.666472,17.545936,7.993908e+03,1.323379e+04,8.241089e+04,180.967667,...,4.928898,3.663011,7.456360,0.008284,0.008381,0.129743,6.468360,9.164262,0.011126,0.550600
std,4.710444,25.348181,2.642982,1.087298,133.916353,115.574086,1.716423e+05,1.514715e+05,1.486204e+05,101.513358,...,8.389545,5.915386,11.415191,0.091171,0.092485,0.638683,8.543927,11.121413,0.104891,0.497436
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.400000e+01,0.000000e+00,0.000000e+00,0.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,0.000008,0.000000,0.000000,0.000000,2.000000,0.000000,1.140000e+02,0.000000e+00,2.860611e+01,62.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000
50%,0.014138,2.000000,0.000000,1.000000,6.000000,2.000000,5.340000e+02,1.780000e+02,2.650177e+03,254.000000,...,1.000000,1.000000,3.000000,0.000000,0.000000,0.000000,3.000000,5.000000,0.000000,1.000000
75%,0.719360,2.000000,6.000000,1.000000,12.000000,10.000000,1.280000e+03,9.560000e+02,1.111111e+05,254.000000,...,4.000000,3.000000,6.000000,0.000000,0.000000,0.000000,7.000000,11.000000,0.000000,1.000000
max,59.999989,130.000000,12.000000,6.000000,10646.000000,11018.000000,1.435577e+07,1.465753e+07,1.000000e+06,255.000000,...,59.000000,38.000000,63.000000,2.000000,2.000000,16.000000,60.000000,62.000000,1.000000,1.000000


In [11]:
# Select ultra_3_short based features and labels (column 43)
train_features= selected_data[RFE_feature_list]
#labels= selected_data.iloc[:,selected_data.shape[1]-1:]
attack_labels=selected_data['label']
print('features shape is:', train_features.shape, 'labels shape is:', attack_labels.shape)
attack_labels.head(10)

features shape is: (82332, 24) labels shape is: (82332,)


id
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
Name: label, dtype: int64

In [12]:
# Separate data in train set and test set
df= pd.DataFrame(train_features)
# create training and testing vars
# Note: train_size + test_size < 1.0 means we are subsampling
# Use small numbers for slow classifiers, as KNN, Radius, SVC,...
X_train, X_test, y_train, y_test = train_test_split(df, attack_labels, train_size=0.8, test_size=0.2, random_state=1)
print('X_train, y_train:', X_train.shape, y_train.shape)
print('X_test, y_test:', X_test.shape, y_test.shape)

X_train, y_train: (65865, 24) (65865,)
X_test, y_test: (16467, 24) (16467,)


In [13]:
# Training, choose model by commenting/uncommenting clf= Random Forest
print('Training model...')
clf= RandomForestClassifier(n_jobs=-1, random_state=3, n_estimators=102)
#, max_features=0.8, min_samples_leaf=3, n_estimators=500, min_samples_split=3, random_state=10, verbose=1)
# clf = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, presort=False)
# clf= svm.SVC(kernel='rbf')
#clf= neighbors.KNeighborsClassifier(n_neighbors=1, algorithm='ball_tree', metric='manhattan')

trained_model= clf.fit(X_train, y_train)

# Predicting
print('Predicting...')
y_pred = clf.predict(X_test)
#print('Predited labels: \t', y_pred)
#print('True labels: \t\t', y_test)

results = confusion_matrix(y_test, y_pred)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(y_test, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(y_test,y_pred))


Training model...
Predicting...

Confusion matrix:
 [[7198  140]
 [ 203 8926]]
Computing performance metrics

Accuracy result: 0.979170462136394

Classification report:
             precision    recall  f1-score   support

          0       0.97      0.98      0.98      7338
          1       0.98      0.98      0.98      9129

avg / total       0.98      0.98      0.98     16467



In [14]:
# Training, choose model by commenting/uncommenting clf= KNN = 1
print('Training model...')
clf= neighbors.KNeighborsClassifier(n_neighbors=1, algorithm='ball_tree', metric='manhattan')

trained_model= clf.fit(X_train, y_train)

# Predicting
print('Predicting...')
y_pred = clf.predict(X_test)
#print('Predited labels: \t', y_pred)
#print('True labels: \t\t', y_test)

results = confusion_matrix(y_test, y_pred)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(y_test, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(y_test,y_pred))

Training model...
Predicting...

Confusion matrix:
 [[6693  645]
 [ 762 8367]]
Computing performance metrics

Accuracy result: 0.9145563854982692

Classification report:
             precision    recall  f1-score   support

          0       0.90      0.91      0.90      7338
          1       0.93      0.92      0.92      9129

avg / total       0.91      0.91      0.91     16467



In [15]:
# Training, choose model by commenting/uncommenting clf= KNN = 3
print('Training model...')
clf= neighbors.KNeighborsClassifier(n_neighbors=3, algorithm='ball_tree', metric='manhattan')

trained_model= clf.fit(X_train, y_train)

# Predicting
print('Predicting...')
y_pred = clf.predict(X_test)
print('Predited labels: \t', y_pred)
#print('True labels: \t\t', y_test)

results = confusion_matrix(y_test, y_pred)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(y_test, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(y_test,y_pred))

Training model...
Predicting...
Predited labels: 	 [0 1 0 ... 1 0 1]

Confusion matrix:
 [[6775  563]
 [ 809 8320]]
Computing performance metrics

Accuracy result: 0.916681848545576

Classification report:
             precision    recall  f1-score   support

          0       0.89      0.92      0.91      7338
          1       0.94      0.91      0.92      9129

avg / total       0.92      0.92      0.92     16467



In [16]:
# Training, choose model by commenting/uncommenting clf= KNN = 5
print('Training model...')
clf= neighbors.KNeighborsClassifier(n_neighbors=5, algorithm='ball_tree', metric='manhattan')

trained_model= clf.fit(X_train, y_train)

# Predicting
print('Predicting...')
y_pred = clf.predict(X_test)
#print('Predited labels: \t', y_pred)
#print('True labels: \t\t', y_test)

results = confusion_matrix(y_test, y_pred)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(y_test, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(y_test,y_pred))

Training model...
Predicting...

Confusion matrix:
 [[6818  520]
 [ 834 8295]]
Computing performance metrics

Accuracy result: 0.917774943827048

Classification report:
             precision    recall  f1-score   support

          0       0.89      0.93      0.91      7338
          1       0.94      0.91      0.92      9129

avg / total       0.92      0.92      0.92     16467



In [17]:
# Training, choose model by commenting/uncommenting clf= GMM = 2
print('Training model...')
clf= GaussianMixture(n_components = 2, covariance_type = 'full')

trained_model= clf.fit(train_features)

# Predicting
print('Predicting...')
y_pred = clf.predict(train_features)
print('Predited labels: \t', y_pred)
for i, c in enumerate(y_pred):
    if c != 0:
        y_pred[i]= 1
print('Predited labels: \t', y_pred)        
#print('True labels: \t\t', y_test)

results = confusion_matrix(attack_labels, y_pred)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(attack_labels, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(attack_labels,y_pred))

Training model...
Predicting...
Predited labels: 	 [0 0 0 ... 0 0 0]
Predited labels: 	 [0 0 0 ... 0 0 0]

Confusion matrix:
 [[26058 10942]
 [45332     0]]
Computing performance metrics

Accuracy result: 0.3164990526162367

Classification report:
             precision    recall  f1-score   support

          0       0.37      0.70      0.48     37000
          1       0.00      0.00      0.00     45332

avg / total       0.16      0.32      0.22     82332



In [18]:
# Training, choose model by commenting/uncommenting clf= GMM = 10
print('Training model...')
clf= GaussianMixture(n_components = 10, covariance_type = 'full')

trained_model= clf.fit(train_features)

# Predicting
print('Predicting...')
y_pred = clf.predict(train_features)
print('Predited labels: \t', y_pred)
for i, c in enumerate(y_pred):
    if c != 0:
        y_pred[i]= 1
print('Predited labels: \t', y_pred)        
#print('True labels: \t\t', y_test)

results = confusion_matrix(attack_labels, y_pred)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(attack_labels, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(attack_labels,y_pred))

Training model...
Predicting...
Predited labels: 	 [0 0 0 ... 0 0 0]
Predited labels: 	 [0 0 0 ... 0 0 0]

Confusion matrix:
 [[ 8691 28309]
 [30648 14684]]
Computing performance metrics

Accuracy result: 0.2839114803478599

Classification report:
             precision    recall  f1-score   support

          0       0.22      0.23      0.23     37000
          1       0.34      0.32      0.33     45332

avg / total       0.29      0.28      0.29     82332



In [19]:
# Training, choose model by commenting/uncommenting clf= K-Means = 2
from sklearn import cluster
print('Training model...')
clf= cluster.KMeans(n_clusters = 2)

trained_model= clf.fit(X_train, y_train)

# Predicting
print('Predicting...')
y_pred = clf.predict(X_test)
#print('Predited labels: \t', y_pred)
#print('True labels: \t\t', y_test)

print('Predited labels: \t', y_pred)
for i, c in enumerate(y_pred):
    if c != 0:
        y_pred[i]= 1
print('Predited labels: \t', y_pred)        
#print('True labels: \t\t', y_test)

results = confusion_matrix(y_test, y_pred)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(y_test, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(y_test,y_pred))

Training model...
Predicting...
Predited labels: 	 [0 0 0 ... 0 0 0]
Predited labels: 	 [0 0 0 ... 0 0 0]

Confusion matrix:
 [[6689  649]
 [9129    0]]
Computing performance metrics

Accuracy result: 0.40620635209813566

Classification report:
             precision    recall  f1-score   support

          0       0.42      0.91      0.58      7338
          1       0.00      0.00      0.00      9129

avg / total       0.19      0.41      0.26     16467



In [20]:
# Training, choose model by commenting/uncommenting clf= K-Means = 10
from sklearn import cluster
print('Training model...')
clf= cluster.KMeans(n_clusters = 10)

trained_model= clf.fit(X_train, y_train)

# Predicting
print('Predicting...')
y_pred = clf.predict(X_test)
#print('Predited labels: \t', y_pred)
#print('True labels: \t\t', y_test)

print('Predited labels: \t', y_pred)
for i, c in enumerate(y_pred):
    if c != 0:
        y_pred[i]= 1
print('Predited labels: \t', y_pred)        
#print('True labels: \t\t', y_test)

results = confusion_matrix(y_test, y_pred)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(y_test, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(y_test,y_pred))

Training model...
Predicting...
Predited labels: 	 [0 0 0 ... 0 0 0]
Predited labels: 	 [0 0 0 ... 0 0 0]

Confusion matrix:
 [[5029 2309]
 [8988  141]]
Computing performance metrics

Accuracy result: 0.3139612558450234

Classification report:
             precision    recall  f1-score   support

          0       0.36      0.69      0.47      7338
          1       0.06      0.02      0.02      9129

avg / total       0.19      0.31      0.22     16467



In [21]:
from sklearn.ensemble import IsolationForest

# Training, choose model by commenting/uncommenting clf= Random Isolation Forest

print('Training model...')
clf = IsolationForest(random_state=42)

trained_model= clf.fit(X_train, y_train)

# Predicting
print('Predicting...')
y_pred = clf.predict(X_test)
#print('Predited labels: \t', y_pred)
#print('True labels: \t\t', y_test)

print('Predited labels: \t', y_pred)
for i, c in enumerate(y_pred):
    if c > 0:
        y_pred[i]= 1
    else:
        y_pred[i]= 0
print('Predited labels: \t', y_pred)        
#print('True labels: \t\t', y_test)

results = confusion_matrix(y_test, y_pred)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(y_test, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(y_test,y_pred))

Training model...


/home/will/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


Predicting...
Predited labels: 	 [ 1 -1  1 ...  1  1 -1]
Predited labels: 	 [1 0 1 ... 1 1 0]

Confusion matrix:
 [[ 859 6479]
 [ 773 8356]]
Computing performance metrics

Accuracy result: 0.5596040565980446

Classification report:
             precision    recall  f1-score   support

          0       0.53      0.12      0.19      7338
          1       0.56      0.92      0.70      9129

avg / total       0.55      0.56      0.47     16467



In [22]:
#  clf= Class MLPClassifier implements a multi-layer perceptron (MLP) 
# algorithm that trains using Backpropagation.
from sklearn.neural_network import MLPClassifier

print('Training model...')
clf = MLPClassifier(solver='adam', random_state=42)

trained_model= clf.fit(X_train, y_train)

# Predicting
print('Predicting...')
y_pred = clf.predict(X_test)
print('Predited labels: \t', y_pred)
#print('True labels: \t\t', y_test)

results = confusion_matrix(y_test, y_pred)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(y_test, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(y_test,y_pred))


Training model...
Predicting...
Predited labels: 	 [0 1 0 ... 0 0 1]

Confusion matrix:
 [[6634  704]
 [3016 6113]]
Computing performance metrics

Accuracy result: 0.7740936418291128

Classification report:
             precision    recall  f1-score   support

          0       0.69      0.90      0.78      7338
          1       0.90      0.67      0.77      9129

avg / total       0.80      0.77      0.77     16467



In [23]:
# Training, choose model by commenting/uncommenting clf= SVC - rbf kernel
print('Training model...')
# clf= RandomForestClassifier(n_jobs=-1, random_state=3, n_estimators=102)
#, max_features=0.8, min_samples_leaf=3, n_estimators=500, min_samples_split=3, random_state=10, verbose=1)
# clf = DecisionTreeClassifier(criterion='gini', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, class_weight=None, presort=False)
clf= svm.SVC(kernel='rbf')
#clf= neighbors.KNeighborsClassifier(n_neighbors=1, algorithm='ball_tree', metric='manhattan')

trained_model= clf.fit(X_train, y_train)

# Predicting
print('Predicting...')
y_pred = clf.predict(X_test)
#print('Predited labels: \t', y_pred)
#print('True labels: \t\t', y_test)

results = confusion_matrix(y_test, y_pred)
print('\nConfusion matrix:\n', results)

print('Computing performance metrics')
print('\nAccuracy result:', accuracy_score(y_test, y_pred, normalize=True))


print("\nClassification report:")
print(classification_report(y_test,y_pred))


Training model...
Predicting...

Confusion matrix:
 [[7297   41]
 [2805 6324]]
Computing performance metrics

Accuracy result: 0.8271694904961439

Classification report:
             precision    recall  f1-score   support

          0       0.72      0.99      0.84      7338
          1       0.99      0.69      0.82      9129

avg / total       0.87      0.83      0.83     16467

